# MWRTM Evaluation

Quick evaluation of MWRTM component performance (mainly FAP and Jacobian, also background temperature adjustment)

In [ ]:
from collections import OrderedDict
from functools import partial
import random
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import mwrt
import mwrt.evaluation as meval
from mwrt.fapgen import absorption_model
from faps_hatpro import *
from db_tools import Database

%matplotlib inline

In [ ]:
db = Database("../data/amalg.db")

In [ ]:
FAPs = [FAP22240MHz, FAP23040MHz, FAP23840MHz, FAP25440MHz, FAP26240MHz, FAP27840MHz, FAP31400MHz,
        FAP51260MHz, FAP52280MHz, FAP53860MHz, FAP54940MHz, FAP56660MHz, FAP57300MHz, FAP58000MHz]

FAPnames = ["TB_22240MHz", "TB_23040MHz", "TB_23840MHz", "TB_25440MHz", "TB_26240MHz", "TB_27840MHz", "TB_31400MHz",
        "TB_51260MHz", "TB_52280MHz", "TB_53860MHz", "TB_54940MHz", "TB_56660MHz", "TB_57300MHz", "TB_58000MHz"]

In [ ]:
def get_fap_freq(fap):
    return int(fap.__name__[3:8])/1000

In [ ]:
def get_profile(where):
    df = db.as_dataframe("SELECT z, p, T, qvap, qliq FROM profiledata WHERE {};".format(where)).dropna(axis=0)
    lnq = pd.Series(np.log(df["qvap"] + df["qliq"]), index=df.index, name="lnq")
    lnq[lnq<-100] = -100
    return pd.concat([df, lnq], axis=1)

In [ ]:
def relerr(x, y):
    return np.abs((x - y)/x)*100

## FAP Performance

Compare FAP to full absorption.

In [ ]:
absorp = absorption_model(mwrt.liebe93.refractivity_gaseous, mwrt.tkc.refractivity_lwc)

In [ ]:
df = get_profile("""z > 500 AND p > 100 AND p < 200 LIMIT 10000""")

In [ ]:
df = db.as_dataframe("SELECT p, T, qvap, qliq FROM profiledata WHERE z > 500 AND p > 110 AND p < 200 LIMIT 10000;").dropna(axis=0)
lnq = pd.Series(np.log(df["qvap"] + df["qliq"]), index=df.index, name="lnq")
df = pd.concat([df[["p", "T"]], lnq], axis=1)

In [ ]:
coefs = OrderedDict()
for fap in FAPs:
    ν = get_fap_freq(fap)
    approx = fap(df["p"].values, df["T"].values, df["lnq"].values)
    exact = absorp(ν, df["p"].values, df["T"].values, df["lnq"].values)
    coefs[ν] = (approx, exact)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
err_pos = np.arange(14) - 0.2
val_pos = np.arange(14) + 0.2
names = list(coefs.keys())

ax.boxplot([x[0]/max(x[1]) for x in coefs.values()], positions=err_pos, widths=0.3, showfliers=False, whis=[0, 90])
ax.boxplot([x[1]/max(x[1]) for x in coefs.values()], positions=val_pos, widths=0.3, showfliers=False, whis=[0, 90])
ax.set_xticklabels(names)
ax.set_xlim(-0.5, 13.5)

## Model Jacobian

Compare autodiff Jacobian to finite differencing Jacobian.

In [ ]:
dry_profiles = list(x[0] for x in db.execute("""select id from profiles where cloudy = 0 and kind = "cosmo7";"""))
cloudy_profiles = list(x[0] for x in db.execute("""select id from profiles where cloudy = 1 and kind = "cosmo7";"""))

test_profiles = random.sample(dry_profiles, 3) + random.sample(dry_profiles, 3)

In [ ]:
model_grid = mwrt.atanspace(1100., 21000., 3000)

In [ ]:
angle = 0.
results = []

for pid in test_profiles:
    df = get_profile("profile = {}".format(pid))
    z = df["z"].values
    p = df["p"].values
    T = df["T"].values
    lnq = df["lnq"].values
    itp = mwrt.LinearInterpolation(source=z, target=model_grid)
    model = mwrt.MWRTM(itp, FAP52280MHz)
    results.append(meval.fd_jacobian(model, angle, p, T, lnq, perturbation=0.0001))

In [ ]:
for bt, dt, dlnq in results:
    print("{:>10.8f}   {:>10.8f}".format(np.max(relerr(bt.dT, dt)), np.max(relerr(bt.dlnq, dlnq))))

In [ ]:
%timeit -n50 model(angles=angle, data=df)
%timeit -n50 model.forward(angle, data=df)
pass

In [ ]:
model1 = mwrt.MWRTM(itp, FAP52280MHz, background=2.75)
model2 = mwrt.MWRTM(itp, FAP52280MHz, background=5.708)

In [ ]:
model1.forward(0., data=df), model2.forward(0., data=df, )

## Cosmic Background Adjustment

Accounts for stratospheric emission.

In [ ]:
from mwrt.background import USStandardBackground
from mwrt.fapgen import as_absorption
import mwrt

In [ ]:
uss = USStandardBackground(15000, 32000, 120, n=5000)

In [ ]:
uss.evaluate(partial(as_absorption(mwrt.liebe93.refractivity_gaseous), 52.28))